<a href="https://colab.research.google.com/github/prof-nuduls/EE628HW/blob/main/AAI627HW8Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

In [3]:
!tar xf spark-3.5.1-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [5]:
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [6]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [19]:
training = spark.read.csv("re_u.data")
training.show(5)

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|196|242|  3|
|186|302|  3|
| 22|377|  1|
|244| 51|  2|
|166|346|  1|
+---+---+---+
only showing top 5 rows



In [20]:
training = training.withColumnRenamed("_c0", "item1").withColumnRenamed("_c1", "item2").withColumnRenamed("_c2", "target")
training.show(5)

+-----+-----+------+
|item1|item2|target|
+-----+-----+------+
|  196|  242|     3|
|  186|  302|     3|
|   22|  377|     1|
|  244|   51|     2|
|  166|  346|     1|
+-----+-----+------+
only showing top 5 rows



In [21]:
from pyspark.sql.types import IntegerType
training = training.withColumn("item1", training["item1"].cast(IntegerType()))
training = training.withColumn("item2", training["item2"].cast(IntegerType()))
training = training.withColumn("target", training["target"].cast(IntegerType()))
training.show(3)

+-----+-----+------+
|item1|item2|target|
+-----+-----+------+
|  196|  242|     3|
|  186|  302|     3|
|   22|  377|     1|
+-----+-----+------+
only showing top 3 rows



In [29]:
rank = [5,7,9,10,20]
mse = []
for ranks in rank:
  # Create ALS model
  als = ALS(
      maxIter=20,
      rank = ranks,
      regParam=0.01,
      userCol="item1",
      itemCol="item2",
      ratingCol="target",
      nonnegative = True,
      implicitPrefs = False,
      coldStartStrategy="drop"
  )
  model = als.fit(training)

  # Evaluate the model by computing the RMSE on the test data
  predictions = model.transform(training)
  evaluator = RegressionEvaluator(metricName="mse", labelCol="target",
                                  predictionCol="prediction")
  mse = evaluator.evaluate(predictions)
  print("mean-square error = " + str(mse))

mean-square error = 0.626975934375562
mean-square error = 0.5728014676518232
mean-square error = 0.5288615466457607
mean-square error = 0.5063126844287225
mean-square error = 0.3621487589349463


In [30]:
iter = [2,5,7,10]
mse = []
for maxiter in iter:
  # Create ALS model
  als = ALS(
      maxIter=maxiter,
      rank = 10,
      regParam=0.01,
      userCol="item1",
      itemCol="item2",
      ratingCol="target",
      nonnegative = True,
      implicitPrefs = False,
      coldStartStrategy="drop"
  )
  model = als.fit(training)

  # Evaluate the model by computing the RMSE on the test data
  predictions = model.transform(training)
  evaluator = RegressionEvaluator(metricName="mse", labelCol="target",
                                  predictionCol="prediction")
  mse = evaluator.evaluate(predictions)
  print("mean-square error = " + str(mse))

mean-square error = 0.7459878002533074
mean-square error = 0.5616903950679217
mean-square error = 0.5406751657221828
mean-square error = 0.5250503372171159


In [36]:
amounts = [2000,5000,10000,20000,50000,100000]
mse = []
for amount in amounts:
  # Create ALS model
  als = ALS(
      maxIter=20,
      rank = 10,
      regParam=0.01,
      userCol="item1",
      itemCol="item2",
      ratingCol="target",
      nonnegative = True,
      implicitPrefs = False,
      coldStartStrategy="drop"
  )
  model = als.fit(training.limit(amount))

  # Evaluate the model by computing the RMSE on the test data
  predictions = model.transform(training)
  evaluator = RegressionEvaluator(metricName="mse", labelCol="target",
                                  predictionCol="prediction")
  mse = evaluator.evaluate(predictions)
  print("mean-square error = " + str(mse))

mean-square error = 2.6380702020063582
mean-square error = 2.0266502369594663
mean-square error = 1.571747447807223
mean-square error = 1.103903673787555
mean-square error = 0.7494254160255109
mean-square error = 0.5063126844286863
